In [1]:
%pip install geopandas shapely rtree tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
 %pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point
from tqdm import tqdm
from scipy.spatial import cKDTree
import numpy as np

# === CONFIG ===
FACILITY_FILE = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\master_dataset_final.csv"
SNAPPED_VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_villages_by_district"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Load facilities
facilities_df = pd.read_csv(FACILITY_FILE)
districts = sorted(facilities_df["Districtname"].dropna().str.lower().unique())

In [2]:
def snap_facilities_to_nearest_road(village_path, facility_df, district_name):
    # Load snapped village file for reference
    village_gdf = gpd.read_file(village_path)
    village_gdf = village_gdf.to_crs(epsg=4326)

    # Prepare KD-Tree for fast nearest neighbor lookup
    village_coords = np.array(list(zip(village_gdf.geometry.x, village_gdf.geometry.y)))
    tree = cKDTree(village_coords)

    # Filter facilities for this district
    df = facility_df[facility_df["Districtname"].str.lower() == district_name]
    df = df.dropna(subset=["longitude", "latitude", "NIN"])

    snapped_rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Snapping facilities in {district_name}"):
        point = (row["longitude"], row["latitude"])
        dist, idx = tree.query(point)
        snapped_point = village_coords[idx]
        geom = Point(snapped_point)

        row_data = row.to_dict()
        row_data["snapped_lon"] = snapped_point[0]
        row_data["snapped_lat"] = snapped_point[1]
        row_data["geometry"] = geom
        snapped_rows.append(row_data)

    snapped_gdf = gpd.GeoDataFrame(snapped_rows, geometry="geometry", crs="EPSG:4326")
    return snapped_gdf

In [3]:
for district in tqdm(districts, desc="🔁 Snapping All Districts"):
    village_file = os.path.join(SNAPPED_VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    if not os.path.exists(village_file):
        print(f"❌ Missing: {district}")
        continue

    gdf = snap_facilities_to_nearest_road(village_file, facilities_df, district)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_facilities_snapped.geojson")
    gdf.to_file(out_path, driver="GeoJSON")
    print(f"✅ Saved: {out_path}")

🔁 Snapping All Districts:   3%|▎         | 1/30 [00:01<00:43,  1.52s/it]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district\anugul_facilities_snapped.geojson


🔁 Snapping All Districts:   7%|▋         | 2/30 [00:01<00:22,  1.23it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district\balangir_facilities_snapped.geojson


🔁 Snapping All Districts:  10%|█         | 3/30 [00:02<00:17,  1.58it/s]

...

Snapping facilities in sonepur: 100%|██████████| 114/114 [00:00<00:00, 1401.10it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district\sonepur_facilities_snapped.geojson

🔁 Snapping All Districts:  97%|█████████▋| 29/30 [00:13<00:00,  3.14it/s]

🔁 Snapping All Districts: 100%|██████████| 30/30 [00:13<00:00,  2.22it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district\sundargarh_facilities_snapped.geojson


In [4]:
import geopandas as gpd
import pandas as pd

# 📂 Path to the Anugul snapped facilities file
file_path = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Data\snapped_facilities_by_district\anugul_facilities_snapped.geojson"

# ✅ Load the GeoJSON
gdf = gpd.read_file(file_path)

# 📋 Summary checks
print("✅ Total snapped facilities:", len(gdf))

# 🔍 Check for missing or duplicate NINs
missing_nin = gdf['NIN'].isnull().sum()
duplicate_nin = gdf[gdf.duplicated(subset='NIN', keep=False)]

print("❌ Missing NINs:", missing_nin)
print("⚠️ Duplicate NINs:", duplicate_nin['NIN'].nunique())

# 📍 Validate geometry
invalid_geom = gdf[~gdf.geometry.is_valid | gdf.geometry.is_empty]
print("❌ Invalid geometries:", len(invalid_geom))

# 📑 Quick preview
print("\n📌 Sample entries:")
print(gdf[['NIN', 'Facilityname', 'Districtname', 'facility_type_standardized', 'geometry']].head())

✅ Total snapped facilities: 210
❌ Missing NINs: 0
⚠️ Duplicate NINs: 1
❌ Invalid geometries: 0

📌 Sample entries:
          NIN     Facilityname Districtname facility_type_standardized  \
0  6728864528          Bantala       Anugul                        chc   
1  7324817373         MADHAPUR       Anugul                        chc   
2  7352515139         BANARPAL       Anugul                        chc   
3  5687542687  RAJKISHORENAGAR       Anugul                        chc   
4  8282723678           Kosala       Anugul                        chc   

                    geometry  
0  POINT (85.07392 20.73864)  
1  POINT (84.42443 20.78356)  
2  POINT (85.21591 20.84181)  
3   POINT (84.4844 20.95473)  
4   POINT (84.9333 21.01649)  
